__Author__: Bogdan Bintu

__Email__: bbintu@g.harvard.edu

__Date__:3/4/2020

__Platform__: Python 2.7

In [ ]:
import cPickle as pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist,cdist,squareform
from tqdm import tqdm_notebook as tqdm
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.size']=15
matplotlib.rcParams['font.family']='Arial'
import matplotlib.pyplot as plt
import numpy as np
import os

#### Please specifiy the data/save folders

In [ ]:
data_folder = r'\data' #This is the folder containing the .tsv data files
save_data = r'\data'

### 1. Calculate the normalized proximity frequency for A/B loci not associated with speckles

#### Load the positions of the chromatin loci imaged

In [ ]:
folder = data_folder
experiment = []
fid = open(folder+os.sep+r'genomic-scale-with transcription and nuclear bodies.tsv','r')
lines = np.array([ln[:-1].split('\t')for ln in fid if len(ln)>0])
head = list(lines[0])
experiment = np.concatenate([experiment,lines[1::2082,head.index('experiment number')].astype(int)])
zxy = np.array(lines[1:,:3][:],dtype=np.float)
zxy = zxy.reshape([-1,2082,3])/1000 #transform to um

In [ ]:
tot_distsSC_WT_ = np.array(lines[1:,-1],dtype=np.float).reshape([-1,2082])/1000
makenan_nSC = (tot_distsSC_WT_<0.25)|np.isnan(tot_distsSC_WT_)

In [ ]:
lens = [76, 80, 66, 63, 60, 55, 53, 48, 40, 43, 44, 44, 33, 30, 31, 30, 33, 33, 33, 33, 31, 31, 51]
edges = [0]+list(np.cumsum(lens))
ijs = []
fl_ = save_data+r'/mat_contact_IMR90_untreated_non-speckle.pkl'
makenan = makenan_nSC
for i in range(len(lens)):
    for j in range(len(lens)):
        ijs.append((i,j))
        
im_med = np.zeros([edges[-1],edges[-1]])
cut_offs = [0.25,0.5,0.75,1]
im_fr = np.zeros([edges[-1],edges[-1],len(cut_offs)])
im_med_trans = []
im_med_cis = []
im_fr_trans = [[] for _ in cut_offs]
im_fr_cis = [[] for _ in cut_offs]

zxy_ = zxy.copy()
zxy_[makenan]=np.nan #make nan the elements associated with the speckle

for i,j in tqdm(ijs):
    arr = []
    for st1 in [0,edges[-1]]:
        for st2 in [0,edges[-1]]:
            zxy1 = zxy_[:,st1+edges[i]:st1+edges[i+1]]
            zxy2 = zxy_[:,st2+edges[j]:st2+edges[j+1]]
            arr =arr+[cdist(zxy1[k],zxy2[k]) for k in range(len(zxy1))]
    arr = np.array(arr)
    im_med[edges[i]:edges[i+1],edges[j]:edges[j+1]]=np.nanmedian(arr,axis=0)
    if i==j:
        im_med_cis.append(np.nanmedian(arr[::2],axis=0))
        im_med_trans.append(np.nanmedian(arr[1::2],axis=0))
    for ic,cutoff in enumerate(cut_offs):
        im_fr[edges[i]:edges[i+1],edges[j]:edges[j+1],ic] = 1.*np.sum(arr<cutoff,0)/np.sum(arr>-1,0)
        if i==j:
            im_fr_trans[ic].append(1.*np.sum(arr[1::2]<cutoff,0)/np.sum(arr[1::2]>-1,0))
            im_fr_cis[ic].append(1.*np.sum(arr[::2]<cutoff,0)/np.sum(arr[::2]>-1,0))

pickle.dump([im_med,im_fr,im_med_trans,im_med_cis,im_fr_trans,im_fr_cis,len(zxy_)],
        open(fl_,'wb'))
        

In [ ]:
im_med,im_fr,im_med_trans,im_med_cis,im_fr_trans,im_fr_cis,nlen=  pickle.load(
    open(save_data+r'/mat_contact_IMR90_untreated_non-speckle.pkl','rb'))

In [ ]:
#Number of regions/chromosome
lens = [76, 80, 66, 63, 60, 55, 53, 48, 40, 43, 44, 44, 33, 30, 31, 30, 33, 33, 33, 33, 31, 31, 51]
edges = np.cumsum([0]+lens)
#AB identity
AB = 'B,B,A,A,B,B,A,A,A,B,A,A,A,B,A,B,B,A,B,B,B,B,B,B,B,A,B,B,A,A,A,B,B,B,B,B,B,B,B,A,nan,A,A,A,B,A,B,A,B,A,B,A,B,A,A,A,B,B,B,A,A,A,B,B,A,B,B,A,B,B,B,B,B,B,B,A,B,B,A,A,B,B,B,A,A,B,A,B,A,A,B,B,B,A,B,B,A,B,A,B,A,B,B,B,B,B,nan,A,B,A,B,B,A,B,B,A,B,B,B,B,A,B,B,A,B,A,B,B,A,B,B,A,A,A,B,B,A,B,A,A,B,B,A,B,B,B,B,A,A,B,A,B,A,B,B,A,B,B,B,B,A,B,B,A,B,A,A,B,B,A,A,A,B,B,A,B,B,A,A,B,B,B,B,B,A,B,nan,B,A,A,B,A,B,A,B,A,A,A,A,B,B,A,B,B,B,A,B,B,B,B,B,A,A,B,A,B,A,A,B,B,A,A,A,B,B,B,A,B,B,A,A,B,B,B,A,A,B,B,nan,A,A,B,B,B,B,B,B,B,B,B,A,B,B,B,A,B,B,B,B,A,B,A,A,A,B,B,B,A,A,B,B,A,B,B,A,B,B,B,B,B,A,B,A,B,A,B,B,A,B,B,B,B,B,B,B,A,B,A,B,B,nan,B,A,A,B,B,A,B,A,B,A,A,A,B,B,A,A,B,B,B,B,B,B,B,B,A,B,B,B,A,A,B,A,B,A,B,B,B,B,B,B,B,B,A,A,A,B,B,A,A,A,A,B,B,A,A,A,B,A,B,B,B,A,A,B,B,B,B,A,B,B,B,B,A,B,B,B,B,B,A,A,B,B,B,B,B,A,A,A,B,A,A,A,A,B,B,B,B,B,B,B,A,B,B,B,B,B,B,B,A,A,A,B,A,A,A,B,B,B,nan,B,A,B,B,A,A,A,A,B,B,A,B,A,A,A,A,B,B,A,B,B,B,A,B,A,A,B,B,B,B,B,B,B,B,B,A,B,B,A,B,B,B,A,B,B,A,A,nan,A,B,A,B,B,B,B,A,A,B,B,A,B,B,B,B,B,A,B,A,B,B,B,B,A,A,B,B,B,B,B,A,nan,B,B,B,B,B,B,B,B,A,B,B,A,B,nan,nan,B,B,B,B,B,B,B,B,B,B,A,A,B,A,B,A,A,B,B,A,A,A,A,B,B,B,A,B,A,A,A,B,B,B,A,A,B,nan,A,nan,A,B,B,B,B,B,A,A,A,A,B,B,A,B,A,B,B,A,B,B,B,B,B,B,B,B,B,B,A,B,A,A,B,B,B,A,B,B,A,A,B,B,B,A,nan,B,B,B,A,A,A,A,A,B,B,B,B,A,A,B,B,A,B,A,B,A,B,A,B,B,B,B,A,A,B,B,B,B,B,B,A,B,B,nan,B,B,B,A,A,A,A,B,B,A,B,B,B,A,B,B,B,A,A,B,B,B,A,B,B,B,B,B,A,B,B,A,nan,A,A,B,B,B,B,B,A,A,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,nan,nan,B,B,B,B,B,B,B,B,B,B,A,A,B,B,B,B,A,B,A,B,B,B,B,B,B,B,B,A,A,nan,nan,B,B,B,B,A,B,A,A,B,A,B,B,B,B,B,A,A,A,B,A,A,B,B,B,A,B,B,B,B,A,B,B,B,B,A,B,B,B,B,A,B,B,nan,B,B,B,A,B,B,B,A,A,B,B,B,B,B,A,A,A,A,A,B,B,B,A,A,B,nan,B,A,B,B,A,A,A,A,A,A,B,B,B,A,A,A,A,B,B,A,A,A,A,B,B,B,A,A,B,nan,nan,A,A,B,B,B,B,A,B,A,B,A,B,B,B,A,A,B,B,B,A,A,B,A,A,A,A,A,A,B,B,A,B,A,B,A,A,B,B,nan,nan,B,B,B,B,B,B,A,A,A,A,A,A,A,B,B,B,B,B,B,A,B,B,B,B,B,B,B,B,B,B,B,nan,nan,nan,A,A,A,B,B,B,B,B,B,A,B,B,B,B,B,B,A,nan,B,B,nan,nan,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,A,B,B,B,B,B,B,A,A,nan,nan,nan,nan,B,A,A,A,A,A,B,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,nan,A,A,A,A,A,A,A,A,A,A,A,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B'
AB=np.array(AB.split(','))
A,B = AB=='A',AB=='B'

#contact probability derived from imaging
logim = np.log(im_fr[:,:,1])
logim_o = logim.copy()
for ic in range(len(edges)-1):
    logim_o[edges[ic]:edges[ic+1],edges[ic]:edges[ic+1]]=np.nan
logim_o[np.isinf(logim_o)]=np.nan


for il in range(len(edges)-1):
    for jl in range(len(edges)-1):
        logim_o[edges[il]:edges[il+1],edges[jl]:edges[jl+1]]-=np.nanmedian(logim_o[edges[il]:edges[il+1],edges[jl]:edges[jl+1]])

logim_ord = np.concatenate([np.concatenate([logim_o[A,:][:,A],logim_o[B,:][:,A]]),
                np.concatenate([logim_o[A,:][:,B],logim_o[B,:][:,B]])],axis=-1)    

fig = plt.figure(figsize=(15,10))
edgeA = np.sum(A)
plt.plot([edgeA,edgeA],[0,len(logim_ord)],'k')
plt.plot([0,len(logim_ord)],[edgeA,edgeA],'k')
plt.imshow(logim_ord,cmap='seismic',vmax=np.log(2.5),vmin=np.log(0.5))#,vmin=-8.25,vmax=-3.5)
plt.colorbar()

In [ ]:
AA = logim_ord[:edgeA,:edgeA].ravel()
AA = AA[~np.isnan(AA)]
fig = plt.figure()
h1 = plt.hist(AA,bins=np.linspace(-1.05,1.05,36),normed=True,alpha=0.7,color='r',label='AA');

AB = logim_ord[edgeA:,:edgeA].ravel()
AB = AB[~np.isnan(AB)]
plt.hist(AB,bins=np.linspace(-1.05,1.05,36),normed=True,alpha=0.75,color='gray',label='AB');

BB = logim_ord[edgeA:,edgeA:].ravel()
BB = BB[~np.isnan(BB)]
h2 = plt.hist(BB,bins=np.linspace(-1.05,1.05,36),normed=True,alpha=0.75,color='b',label='BB');



plt.xlim([-1,1])

np.max(h2[0])
1./np.sum(h2[0])
yvals = [0,0.025,0.05,0.075]
plt.yticks(np.array(yvals)*np.sum(h2[0]),np.array(yvals)*100)

xvals = np.round(np.exp(np.linspace(-1,1,4)),1)
xvals = [0.35,0.7,1.4,2.8]
plt.xticks(np.log(xvals),xvals)
plt.xlabel("Normalized proximity frequency")
plt.ylabel("% of locus pairs")
plt.legend()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(4,12))
labels = ['AA','AB','BB']
data = [np.exp(AA),np.exp(AB),np.exp(BB)]
data = [AA,AB,BB]
bplot = axes.boxplot(data,showfliers=False,patch_artist=True,labels=labels,notch=True,whis =[5,95])
colors = ['red','grey','blue']
for patch, color in zip(bplot['boxes'],colors):
    patch.set_facecolor(color)
ticks_ = [0.35,0.7,1.4,2.8]
plt.yticks(np.log(ticks_),map(str,ticks_))
plt.ylabel('Normalized proximity frequency')
plt.title('Medians AA,AB,BB: '+str(list(np.round([np.exp(np.median(AA)),np.exp(np.median(AB)),np.exp(np.median(BB))],2))))
plt.show()

### 2. Calculate the normalized proximity frequency for A/B loci associated with the lamina

In [ ]:
folder = data_folder

experiment = []
fid = open(folder+os.sep+r'genomic-scale.tsv','r')
lines = np.array([ln[:-1].split('\t')for ln in fid if len(ln)>0])
head = list(lines[0])
experiment = np.concatenate([experiment,lines[1::2082,head.index('experiment number')].astype(int)])
zxy = np.array(lines[1:,:3][:],dtype=np.float)
dLAM = np.array(lines[1:,-1].astype(float))

fid = open(folder+os.sep+r'genomic-scale-with transcription and nuclear bodies.tsv','r')
lines = np.array([ln[:-1].split('\t')for ln in fid if len(ln)>0])
head = list(lines[0])
experiment = np.concatenate([experiment,lines[1::2082,head.index('experiment number')].astype(int)])
dLAM = np.concatenate([dLAM,np.array(lines[1:,-3].astype(float))])
zxy = np.concatenate([zxy,np.array(lines[1:,:3][:],dtype=np.float)])
zxy = zxy.reshape([-1,2082,3])/1000 #transform to um
dLAM = dLAM.reshape([-1,2082])/1000

In [ ]:
makenan_lam = (tot_distsLAM_WT_>0.25)|np.isnan(tot_distsLAM_WT_)

In [ ]:
lens = [76, 80, 66, 63, 60, 55, 53, 48, 40, 43, 44, 44, 33, 30, 31, 30, 33, 33, 33, 33, 31, 31, 51]
edges = [0]+list(np.cumsum(lens))
ijs = []
fl_ = save_data+r'/mat_contact_IMR90_untreated_lamina.pkl'
makenan = makenan_lam
for i in range(len(lens)):
    for j in range(len(lens)):
        ijs.append((i,j))
        
im_med = np.zeros([edges[-1],edges[-1]])
cut_offs = [0.25,0.5,0.75,1]
im_fr = np.zeros([edges[-1],edges[-1],len(cut_offs)])
im_med_trans = []
im_med_cis = []
im_fr_trans = [[] for _ in cut_offs]
im_fr_cis = [[] for _ in cut_offs]

zxy_ = zxy.copy()
zxy_[makenan]=np.nan #make nan the elements associated with the speckle

for i,j in tqdm(ijs):
    arr = []
    for st1 in [0,edges[-1]]:
        for st2 in [0,edges[-1]]:
            zxy1 = zxy_[:,st1+edges[i]:st1+edges[i+1]]
            zxy2 = zxy_[:,st2+edges[j]:st2+edges[j+1]]
            arr =arr+[cdist(zxy1[k],zxy2[k]) for k in range(len(zxy1))]
    arr = np.array(arr)
    im_med[edges[i]:edges[i+1],edges[j]:edges[j+1]]=np.nanmedian(arr,axis=0)
    if i==j:
        im_med_cis.append(np.nanmedian(arr[::2],axis=0))
        im_med_trans.append(np.nanmedian(arr[1::2],axis=0))
    for ic,cutoff in enumerate(cut_offs):
        im_fr[edges[i]:edges[i+1],edges[j]:edges[j+1],ic] = 1.*np.sum(arr<cutoff,0)/np.sum(arr>-1,0)
        if i==j:
            im_fr_trans[ic].append(1.*np.sum(arr[1::2]<cutoff,0)/np.sum(arr[1::2]>-1,0))
            im_fr_cis[ic].append(1.*np.sum(arr[::2]<cutoff,0)/np.sum(arr[::2]>-1,0))

pickle.dump([im_med,im_fr,im_med_trans,im_med_cis,im_fr_trans,im_fr_cis,len(zxy_)],
        open(fl_,'wb'))
        

In [ ]:
im_med,im_fr,im_med_trans,im_med_cis,im_fr_trans,im_fr_cis,nlen=  pickle.load(
    open(save_data+r'/mat_contact_IMR90_untreated_lamina.pkl','rb'))

In [ ]:
#Number of regions/chromosome
lens = [76, 80, 66, 63, 60, 55, 53, 48, 40, 43, 44, 44, 33, 30, 31, 30, 33, 33, 33, 33, 31, 31, 51]
edges = np.cumsum([0]+lens)
#AB identity
AB = 'B,B,A,A,B,B,A,A,A,B,A,A,A,B,A,B,B,A,B,B,B,B,B,B,B,A,B,B,A,A,A,B,B,B,B,B,B,B,B,A,nan,A,A,A,B,A,B,A,B,A,B,A,B,A,A,A,B,B,B,A,A,A,B,B,A,B,B,A,B,B,B,B,B,B,B,A,B,B,A,A,B,B,B,A,A,B,A,B,A,A,B,B,B,A,B,B,A,B,A,B,A,B,B,B,B,B,nan,A,B,A,B,B,A,B,B,A,B,B,B,B,A,B,B,A,B,A,B,B,A,B,B,A,A,A,B,B,A,B,A,A,B,B,A,B,B,B,B,A,A,B,A,B,A,B,B,A,B,B,B,B,A,B,B,A,B,A,A,B,B,A,A,A,B,B,A,B,B,A,A,B,B,B,B,B,A,B,nan,B,A,A,B,A,B,A,B,A,A,A,A,B,B,A,B,B,B,A,B,B,B,B,B,A,A,B,A,B,A,A,B,B,A,A,A,B,B,B,A,B,B,A,A,B,B,B,A,A,B,B,nan,A,A,B,B,B,B,B,B,B,B,B,A,B,B,B,A,B,B,B,B,A,B,A,A,A,B,B,B,A,A,B,B,A,B,B,A,B,B,B,B,B,A,B,A,B,A,B,B,A,B,B,B,B,B,B,B,A,B,A,B,B,nan,B,A,A,B,B,A,B,A,B,A,A,A,B,B,A,A,B,B,B,B,B,B,B,B,A,B,B,B,A,A,B,A,B,A,B,B,B,B,B,B,B,B,A,A,A,B,B,A,A,A,A,B,B,A,A,A,B,A,B,B,B,A,A,B,B,B,B,A,B,B,B,B,A,B,B,B,B,B,A,A,B,B,B,B,B,A,A,A,B,A,A,A,A,B,B,B,B,B,B,B,A,B,B,B,B,B,B,B,A,A,A,B,A,A,A,B,B,B,nan,B,A,B,B,A,A,A,A,B,B,A,B,A,A,A,A,B,B,A,B,B,B,A,B,A,A,B,B,B,B,B,B,B,B,B,A,B,B,A,B,B,B,A,B,B,A,A,nan,A,B,A,B,B,B,B,A,A,B,B,A,B,B,B,B,B,A,B,A,B,B,B,B,A,A,B,B,B,B,B,A,nan,B,B,B,B,B,B,B,B,A,B,B,A,B,nan,nan,B,B,B,B,B,B,B,B,B,B,A,A,B,A,B,A,A,B,B,A,A,A,A,B,B,B,A,B,A,A,A,B,B,B,A,A,B,nan,A,nan,A,B,B,B,B,B,A,A,A,A,B,B,A,B,A,B,B,A,B,B,B,B,B,B,B,B,B,B,A,B,A,A,B,B,B,A,B,B,A,A,B,B,B,A,nan,B,B,B,A,A,A,A,A,B,B,B,B,A,A,B,B,A,B,A,B,A,B,A,B,B,B,B,A,A,B,B,B,B,B,B,A,B,B,nan,B,B,B,A,A,A,A,B,B,A,B,B,B,A,B,B,B,A,A,B,B,B,A,B,B,B,B,B,A,B,B,A,nan,A,A,B,B,B,B,B,A,A,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,nan,nan,B,B,B,B,B,B,B,B,B,B,A,A,B,B,B,B,A,B,A,B,B,B,B,B,B,B,B,A,A,nan,nan,B,B,B,B,A,B,A,A,B,A,B,B,B,B,B,A,A,A,B,A,A,B,B,B,A,B,B,B,B,A,B,B,B,B,A,B,B,B,B,A,B,B,nan,B,B,B,A,B,B,B,A,A,B,B,B,B,B,A,A,A,A,A,B,B,B,A,A,B,nan,B,A,B,B,A,A,A,A,A,A,B,B,B,A,A,A,A,B,B,A,A,A,A,B,B,B,A,A,B,nan,nan,A,A,B,B,B,B,A,B,A,B,A,B,B,B,A,A,B,B,B,A,A,B,A,A,A,A,A,A,B,B,A,B,A,B,A,A,B,B,nan,nan,B,B,B,B,B,B,A,A,A,A,A,A,A,B,B,B,B,B,B,A,B,B,B,B,B,B,B,B,B,B,B,nan,nan,nan,A,A,A,B,B,B,B,B,B,A,B,B,B,B,B,B,A,nan,B,B,nan,nan,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,A,B,B,B,B,B,B,A,A,nan,nan,nan,nan,B,A,A,A,A,A,B,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,nan,A,A,A,A,A,A,A,A,A,A,A,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B'
AB=np.array(AB.split(','))
A,B = AB=='A',AB=='B'

#contact probability derived from imaging
logim = np.log(im_fr[:,:,1])
logim_o = logim.copy()
for ic in range(len(edges)-1):
    logim_o[edges[ic]:edges[ic+1],edges[ic]:edges[ic+1]]=np.nan
logim_o[np.isinf(logim_o)]=np.nan


for il in range(len(edges)-1):
    for jl in range(len(edges)-1):
        logim_o[edges[il]:edges[il+1],edges[jl]:edges[jl+1]]-=np.nanmedian(logim_o[edges[il]:edges[il+1],edges[jl]:edges[jl+1]])

logim_ord = np.concatenate([np.concatenate([logim_o[A,:][:,A],logim_o[B,:][:,A]]),
                np.concatenate([logim_o[A,:][:,B],logim_o[B,:][:,B]])],axis=-1)    

fig = plt.figure(figsize=(15,10))
edgeA = np.sum(A)
plt.plot([edgeA,edgeA],[0,len(logim_ord)],'k')
plt.plot([0,len(logim_ord)],[edgeA,edgeA],'k')
plt.imshow(logim_ord,cmap='seismic',vmax=np.log(2.5),vmin=np.log(0.5))#,vmin=-8.25,vmax=-3.5)
plt.colorbar()

In [ ]:
AA = logim_ord[:edgeA,:edgeA].ravel()
AA = AA[~np.isnan(AA)]
fig = plt.figure()
h1 = plt.hist(AA,bins=np.linspace(-1.05,1.05,36),normed=True,alpha=0.7,color='r',label='AA');

AB = logim_ord[edgeA:,:edgeA].ravel()
AB = AB[~np.isnan(AB)]
plt.hist(AB,bins=np.linspace(-1.05,1.05,36),normed=True,alpha=0.75,color='gray',label='AB');

BB = logim_ord[edgeA:,edgeA:].ravel()
BB = BB[~np.isnan(BB)]
h2 = plt.hist(BB,bins=np.linspace(-1.05,1.05,36),normed=True,alpha=0.75,color='b',label='BB');



plt.xlim([-1,1])

np.max(h2[0])
1./np.sum(h2[0])
yvals = [0,0.025,0.05,0.075]
plt.yticks(np.array(yvals)*np.sum(h2[0]),np.array(yvals)*100)

xvals = np.round(np.exp(np.linspace(-1,1,4)),1)
xvals = [0.35,0.7,1.4,2.8]
plt.xticks(np.log(xvals),xvals)
plt.xlabel("Normalized proximity frequency")
plt.ylabel("% of locus pairs")
plt.legend()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(4,12))
labels = ['AA','AB','BB']
data = [np.exp(AA),np.exp(AB),np.exp(BB)]
data = [AA,AB,BB]
bplot = axes.boxplot(data,showfliers=False,patch_artist=True,labels=labels,notch=True,whis =[5,95])
colors = ['red','grey','blue']
for patch, color in zip(bplot['boxes'],colors):
    patch.set_facecolor(color)
ticks_ = [0.35,0.7,1.4,2.8]
plt.yticks(np.log(ticks_),map(str,ticks_))
plt.ylabel('Normalized proximity frequency')
plt.title('Medians AA,AB,BB: '+str(list(np.round([np.exp(np.median(AA)),np.exp(np.median(AB)),np.exp(np.median(BB))],2))))
plt.show()